# Assignment 1


## BPE tokenizer

You need to implement a BPE tokenizer including both **1.1)** token learning step and **1.2)** tokenizer to tokenize a sentence.

**Grading rules** is based on algorithm correctness and tokenization quality:
1. Algorithm correctness: your code should be logically consistent with BPE introduced in our slides.
2. Token type ratio: Compare the ratio of unique tokens to the total number of tokens. A lower ratio indicates fewer out-of-vocabulary words and better generalization, but very low ratio may lose much information.
3. Subword length distribution: Analyze the distribution of subword lengths. Ideally, the tokenizer should generate a balance of short and long subwords, avoiding too many very short or very long subwords.
4. Running time.

For example, let's consider a simple text: "ChatGPT is an AI developed by OpenAI."

If a tokenizer produces the following tokens:
["Chat", "G", "PT", "is", "an", "AI", "developed", "by", "Open", "AI"]

There are 10 tokens in total and 9 unique tokens (since "AI" appears twice). The token type ratio would be:

```
Token type ratio = Unique tokens / Total tokens = 9 / 10 = 0.9
```
We can calculate the subword length distribution as follows:

Length 1: 1 subwords ("G")
Length 2: 5 subwords ("PT", "is", "an", "AI", "by")
Length 3: 0 subwords ()
Length 4: 2 subword ("Chat", "Open")
Length 9: 1 subword ("developed")

```
weighted mean/std of length : 3.11/2.28
```

First, let prepare dataset! We have provided the files and code below.

1. you need to create a folder named *assignment1* under your google drive folder */content/drive/MyDrive/SMU_MITB_NLP/*.
2. download train and dev files and put them into the assignment folder. File links are given below.
3. run each code block in turn to load dataset.

In [1]:
#@title codes to mount your google drive folder
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_NLP/assignment1/

Mounted at /content/drive
/content/drive/MyDrive/SMU_MITB_NLP/assignment1


In [2]:
#@title import useful packages
import gensim
import pandas as pd
import smart_open

import re, collections
from collections import defaultdict, Counter

import copy, codecs
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator

In [3]:
#@title Utility functions for reading and preprocessing data

def read_corpus(csv_fname, tokens_only=False):
    data = pd.read_csv(csv_fname)
    for sent, tag in zip(data['review'], data['sentiment']):
        tokens = gensim.utils.simple_preprocess(sent)
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, tag)

def get_count_vocab(corpus, tokens_only=False):
    data = []
    if tokens_only:
        for doc in corpus:
            data.extend(doc)
    else:
        for taged_doc in corpus:
            data.extend(taged_doc.words)
    return collections.Counter(data)

### prepare datasets

We provide separated files for [training](https://drive.google.com/file/d/1hP_q5jQSCQpF4SNWXli7jMLOnLKGFAEU/view?usp=sharing), [development](https://drive.google.com/file/d/189hUqCY95Kd_RGJm5DwtckvFiEnk4unh/view?usp=sharing), and testing, respectively, under your google drive folders */content/drive/MyDrive/SMU_MITB_NLP/assignment1/sa_train.csv* and */content/drive/MyDrive/SMU_MITB_NLP/assignment1/sa_dev.csv*. Training file is used to learn your model, development file is used to tune your hyperparameters (e.g., vocabulary size, not for model learning!), and testing file is not provided, which we will use for grading.

You can use the following code to load datasets:

In [4]:
training_corpus = list(read_corpus('./sa_train.csv', tokens_only=False))
dev_corpus = list(read_corpus('./sa_dev.csv', tokens_only=False))
print(training_corpus[:2])

[TaggedDocument(words=['one', 'of', 'the', 'other', 'reviewers', 'has', 'mentioned', 'that', 'after', 'watching', 'just', 'oz', 'episode', 'you', 'll', 'be', 'hooked', 'they', 'are', 'right', 'as', 'this', 'is', 'exactly', 'what', 'happened', 'with', 'me', 'br', 'br', 'the', 'first', 'thing', 'that', 'struck', 'me', 'about', 'oz', 'was', 'its', 'brutality', 'and', 'unflinching', 'scenes', 'of', 'violence', 'which', 'set', 'in', 'right', 'from', 'the', 'word', 'go', 'trust', 'me', 'this', 'is', 'not', 'show', 'for', 'the', 'faint', 'hearted', 'or', 'timid', 'this', 'show', 'pulls', 'no', 'punches', 'with', 'regards', 'to', 'drugs', 'sex', 'or', 'violence', 'its', 'is', 'hardcore', 'in', 'the', 'classic', 'use', 'of', 'the', 'word', 'br', 'br', 'it', 'is', 'called', 'oz', 'as', 'that', 'is', 'the', 'nickname', 'given', 'to', 'the', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'it', 'focuses', 'mainly', 'on', 'emerald', 'city', 'an', 'experimental', 'section', 'of', 'the', 'pr

In [23]:
print(len(training_corpus))
print(len(dev_corpus))

4000
500


training_corpus is a list of [TaggedDocument](https://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.TaggedDocument). You can also access the tokenized document and its corresponding tag as follows:

In [5]:
training_corpus[0].words[:5]

['one', 'of', 'the', 'other', 'reviewers']

In [6]:
# this time won't use the tag information though:)
training_corpus[0].tags

'positive'

In [15]:
get_count_vocab(training_corpus[:2])

Counter({'one': 2,
         'of': 12,
         'the': 32,
         'other': 2,
         'reviewers': 1,
         'has': 3,
         'mentioned': 1,
         'that': 4,
         'after': 1,
         'watching': 3,
         'just': 2,
         'oz': 6,
         'episode': 2,
         'you': 4,
         'll': 3,
         'be': 2,
         'hooked': 1,
         'they': 1,
         'are': 4,
         'right': 2,
         'as': 4,
         'this': 3,
         'is': 12,
         'exactly': 1,
         'what': 2,
         'happened': 1,
         'with': 8,
         'me': 4,
         'br': 12,
         'first': 2,
         'thing': 1,
         'struck': 2,
         'about': 2,
         'was': 3,
         'its': 2,
         'brutality': 1,
         'and': 13,
         'unflinching': 1,
         'scenes': 2,
         'violence': 4,
         'which': 2,
         'set': 1,
         'in': 3,
         'from': 1,
         'word': 2,
         'go': 1,
         'trust': 1,
         'not': 5,
         's



### 1.1 token learning

To implement the token learner for BPE tokenizer. We take the following corpus as an example to illustrate the algorithm briefly.

Corpus:
> “low lower newest newest”,
> “low lower newest widest”,
> “low newest widest”,
> “low newest widest”,
> “low newest longer”.

Algorithm:
1.   Initialize a vocabulary with all characters in the corpus except the end-of-word character, which will be concatenated with a special token (e.g., underscore '_'  here) and then added to the initial vocab, e.g., {'d', 'e', 'g', 'i', 'l', 'n', 'o', 's', 'w’, 't_', 'w_’, 'r_’}.
2.   Find the 2 tokens most frequently adjacent to each other in the corpus, e.g., 'e', 's'.
3.   Add a new merged token to vocabulary, e.g., 'es'.
4.   Replace every adjacent 2 tokens in corpus (step 2) with the merged token in step 3, e.g., ‘e’ ‘s’ -> ‘es’.
5.   Repeated steps 2-4 until a predefined vocabulary size is reached.

### Task
You are to finish a function *learn_bpe* with the following inputs and outputs:

Inputs:
*   *vocab*: {'token_str': count_int, ...}, the word count dictionary of the text corpus.
*   *num_vocab*: integer, the predefined vocabulary size including the initial vocabulary.

Outputs:
*   *result_vocab*: ['token_str', ...], the list of learned vocabulary
*   *result_merge*: ['token1 token2', ...], the list of learned merge operation, where two tokens separated by a whitespace. **Keeping the merge order is important!**

A toy example inputs and outputs are given below for your testing.

Here are some suggestions.

*   You can tune your vocab size *num_vocab* on *dev_corpus*.
*   For adjacent 2 tokens with the same frequency, we merge the token following alphabet order. When using the toy example, both ('e', 's') and ('s', 't_') occur 9 times in the corpus, we merge ('e', 's') to 'es' first, as 'es' is in front of 'st_'. (This is not always required in practice.)
*   You are only allowed to use some neccessary 3rd party packages, such as re, collection, copy. **No tokenizer is allowed**.

In [8]:
#@title toy vocabulary

toy_vocab = {'newest':6, 'low':5, 'widest':3, 'lower':2, 'longer':1}
toy_result_vocab = ['longer_', 'newest_', 'widest_', 'ewest_', 'idest_', 'lower_', 'dest_', 'est_', 'ger_', 'low_', 'er_', 'lon', 'low', 'es', 'ew', 'lo', 'r_', 't_', 'w_',
                    'd', 'e', 'g', 'i', 'l', 'n', 'o', 's', 'w']
toy_result_merge = ['e s', 'es t_', 'l o', 'e w', 'ew est_', 'n ewest_', 'lo w_', 'd est_', 'e r_', 'i dest_', 'w idest_',
                    'lo w', 'low er_', 'g er_', 'lo n', 'lon ger_']

In [9]:
# do not change the code above
# write your code here, you may add more helper functions!
def get_pairs(corpus):
  pairs = defaultdict(int)
  for word, freq in corpus.items():
    # get the list of tokens of each word
    tokens = word.split(' ')
    # get the ferq of different pair
    for i in range(len(tokens) - 1):
      pair = tokens[i] +' ' +  tokens[i + 1]
      pairs[pair] += freq
  return pairs

def update_corpus(pair, corpus):
  new_corpus = {}
  # get the pattern for matching 'token_1 token_2'
  token = pair.replace(' ','')
  pattern = re.compile( pair)

  for word, freq in corpus.items():
    # replace pattern with token. 'a b c' --> 'ab c'
    word_out = pattern.sub(token,word)
    # renew the corpus
    new_corpus[word_out] = corpus[word]
  return new_corpus


# do not change the code below

def sort_bpe_vocab(vocab):
    sorted_vocab = sorted(list(vocab))
    sorted_vocab = sorted(list(sorted_vocab), key=len, reverse=True)
    return sorted_vocab

# total_symbols: true, the number of merges count the initial character; false otherwise. If you don't understand, keep it true.
def learn_bpe(vocab, num_vocab, total_symbols=True):
    result_vocab = set()
    result_merge = []
    # do not change the code above
    # write your code here

    # initialize the vocab with original corpus
    for key in vocab.keys():
      for i,char in enumerate(key):
        if i == len(key) - 1:
          # add special sign for ending character
          result_vocab.add(char + '_')
        else:
          result_vocab.add(char)

      # initialize corpus
      corpus = defaultdict(int)
      for key,v in vocab.items():
        word = ' '.join(key) + '_'
        corpus[word] = v


    while len(result_vocab) < num_vocab:
      pairs=get_pairs(corpus)
      if not pairs:
        break
      # get the most frequent pair
      best_pair = max(pairs,key =pairs.get)
      # combine the pair into a new token
      new_token = best_pair.replace(' ','')
      # add the token to the vocab
      result_vocab.add(new_token)
      # update the corpuse
      corpus = update_corpus(best_pair,corpus)
      # add the merge rule in to the merge list
      result_merge.append(best_pair)

    # do not change the code below
    return sort_bpe_vocab(result_vocab), result_merge

In [10]:
result_vocab, result_merge = learn_bpe(toy_vocab, 50)
print(result_vocab)
print(result_merge)

['longer_', 'newest_', 'widest_', 'lower_', 'est_', 'long', 'low_', 'er_', 'lon', 'low', 'new', 'wid', 'es', 'lo', 'ne', 'r_', 't_', 'w_', 'wi', 'd', 'e', 'g', 'i', 'l', 'n', 'o', 's', 'w']
['e s', 'es t_', 'l o', 'n e', 'ne w', 'new est_', 'lo w_', 'w i', 'wi d', 'wid est_', 'e r_', 'lo w', 'low er_', 'lo n', 'lon g', 'long er_']


In [18]:
corpus = get_count_vocab(training_corpus)
result_vocab, result_merge = learn_bpe(corpus, 1000)
print(result_vocab)
print(result_merge)

['unfortunately_', 'disappointed_', 'entertaining_', 'particularly_', 'performances_', 'relationship_', 'interesting_', 'performance_', 'absolutely_', 'believable_', 'characters_', 'completely_', 'especially_', 'everything_', 'experience_', 'production_', 'themselves_', 'throughout_', 'understand_', 'beautiful_', 'beginning_', 'brilliant_', 'certainly_', 'character_', 'different_', 'excellent_', 'extremely_', 'hilarious_', 'hollywood_', 'obviously_', 'recommend_', 'seriously_', 'something_', 'wonderful_', 'actually_', 'although_', 'american_', 'annoying_', 'anything_', 'audience_', 'children_', 'daughter_', 'dialogue_', 'director_', 'disappoin', 'everyone_', 'favorite_', 'horrible_', 'involved_', 'original_', 'performan', 'possible_', 'probably_', 'remember_', 'supposed_', 'terrible_', 'thinking_', 'together_', 'violence_', 'watching_', 'actress_', 'against_', 'already_', 'amazing_', 'attempt_', 'because_', 'becomes_', 'believe_', 'between_', 'classic_', 'despite_', 'differen', 'effect

### 1.2 BPE Tokenizer (tokenize a sentence)

Congratulations! Now, you have learned a *result_vocab* and *result_merge* as token learner for a BPE tokenizer. This question is to use them to tokenize corpus.

The basic algorithm is briefly given below (suppose we use the toy example above and tokenize the sentence ['newest', 'makes', 'me', 'happy']):

1.   For each word (e.g., 'newest'), initialize tokenized word including the end-of-word token, e.g., 'n e w e s t_'. Repeated the following steps 2-3 until there is no merge operation found or the word is merged back again (e.g., 'newest_').
2.   Get first (most frequent) merge operation in the list of BPE merges that can match any adjacent 2 tokens, e.g., 'e s'.
3.   Update the tokenized word, e.g., 'n e w es t_'.
4.   Check tokenized results to ensure all tokens of a word is in the learned BPE vocabulary, otherwise, replace it with a 'UNK' (adjacent 'UNK' tokens can be replaced with only one 'UNK').
5.   Combine all word token results as the tokenized results of the sentence (e.g., ['newest_', 'UNK', 'e', 'UNK', 'UNK', 'UNK']).

###Task
You are to finish a function *bpe_tokenize* with the following inputs and outputs:

Inputs:
*   *words_in_sentences*: a list of tokens ['token1', 'token2', ...], same format as *dev_corpus[0].words*
*   *bpe_merges*: *result_merge* as obtained above, ['token1 token2', ...], the list of learned merge operation, where two tokens separated by a whitespace. **The merge order reflect the frequency!**
*   *bpe_vocab*: *result_vocab* as obtained above, ['token_str', ...], the list of learned vocabulary.

Outputs:
*   *result_tokens*: ['token1', ...], the list of tokenized results.

In [70]:
#@title bpe tokenizer based on learned vocab
# do not change the code above
# write your code here, you may add more helper functions!
def bpe_merge(word, bpe_merges, bpe_vocab):
    chars = word.split()
    merged = True
    while merged:
        merged = False
        i = 0
        new_chars = []
        while i < len(chars) - 1:
            pair = ' '.join(chars[i:i+2])
            if pair in bpe_merges:
                merged_token = pair.replace(' ', '')
                new_chars.append(merged_token)
                i += 2
                merged = True
            else:
                new_chars.append(chars[i])
                i += 1
        if i == len(chars) - 1:
            new_chars.append(chars[i])
        chars = new_chars

    final_tokens = []
    for char in chars:
        if char in bpe_vocab:
            final_tokens.append(char)
        else:
            if final_tokens and final_tokens[-1] == 'UNK':
                continue
            final_tokens.append('UNK')
    return final_tokens
# do not change the code below

def bpe_tokenize(words_in_sentence, bpe_merges, bpe_vocab):
    result_tokens = []
    # do not change the code above
    # write your code here

    # initialization
    tokens_list = [' '.join(word) + '_' for word in words_in_sentence]

    # merge tokens
    for word in tokens_list:
        tokens = bpe_merge(word, bpe_merges, bpe_vocab)
        result_tokens.extend(tokens)


    # do not change the code below
    return result_tokens


In [71]:
#@title test your tokenizer!
print(bpe_tokenize(['newest', 'makes', 'me', 'happy'], toy_result_merge, toy_result_vocab))

['newest_', 'UNK', 'e', 'UNK', 'UNK', 'UNK']


In [76]:
setence_list = []
for setence in dev_corpus:
  setence_list.extend(setence.words)
result = []
result.extend(bpe_tokenize(setence_list,result_merge,result_vocab))

In [79]:
print(result)
print(len(result))

['this_', 'fe', 'el', 's_', 'as_', 'if_', 'it_', 'is_', 'c', 'z', 'e', 'c', 'h_', 've', 'r', 'si', 'o', 'n_', 'of_', 'pe', 'ar', 'l_', 'ha', 'r', 'bo', 'r_', 'it_', 'ha', 's_', 'same_', 'st', 'or', 'y_', 'bo', 't', 'h_', 'guys_', 'fa', 'll_', 'i', 'n_', 'lo', 've_', 'wi', 't', 'h_', 'the_', 'same_', 'w', 'om', 'a', 'n_', 'and_', 'ad', 'd_', 'to_', 'the_', 'tw', 'is', 't_', 'the_', 'w', 'om', 'a', 'n_', 'is_', 'actually_', 'ma', 'r', 'ri', 'ed_', 'one_', 'wh', 'o', 's', 'e_', 'hus', 'ba', 'n', 'd_', 'ha', 's_', 'be', 'e', 'n_', 'mi', 'ssi', 'n', 'g_', 'fo', 'r_', 'ye', 'a', 'r_', 'do', 'n_', 'th', 'in', 'k_', 'that_', 'the_', 'st', 'or', 'y_', 'li', 'n', 'e_', 'is_', 'to', 'o_', 'st', 'ro', 'n', 'g_', 'the_', 'y', 'ou', 'n', 'ge', 'r_', 'guy_', 'is_', 'qu', 'it', 'e_', 'na', 'u', 'gh', 'ty_', 'that_', 'is_', 'cu', 't', 'e_', 'it_', 'ke', 'pt_', 'me_', 'wa', 't', 'ch', 'ing_', 'be', 'ca', 'u', 's', 'e_', 'of_', 'the_', 'emoti', 'on', 'a', 'l_', 'music_', 'and_', 'the_', 'p', 'le', 'as', 